# *Test Technique Data Engineer*

## *Analyse Ventes PME* 

### *Part 1 : Présentation de nos 3 datasets magasins, produis & ventes*

In [1]:
# On importe la librairie pandas qui va nous servir durant notre manipulation de données
import pandas as pd

#### *Analyse du contenu des datasets*

- Nous allons vérifier les noms des colomnes, les types (object, int64, float64) et les valeurs manquantes (s'il y en a)
- Vérification des colonnes qui devraient etre numériques
- Notre objectif dans un premier temps et d'avoir une compréhension globale des datasets

In [2]:
# On importe nos 3 datesets magasins, produits et ventes
dt_magasins = pd.read_csv("magasins.csv")
dt_produits = pd.read_csv("produits.csv")
dt_ventes = pd.read_csv("ventes.csv")

In [3]:
# Afficher les informations générales du dataset magasins
dt_magasins.info()

# Afficher un aperçu des valeurs uniques par colonne
dt_magasins.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID Magasin          7 non-null      int64 
 1   Ville               7 non-null      object
 2   Nombre de salariés  7 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 296.0+ bytes


ID Magasin            7
Ville                 7
Nombre de salariés    7
dtype: int64

- Chaque magasin est identifié par un identifiant unique
- Les magasins sont localisés par ville et par région
- Ces informations permettront des analyses géographiques

In [4]:
# Analyse du dataset produits
# Afficher les informations générales du dataset produits
dt_produits.info()

# Afficher des statistiques descriptives pour les colonnes numériques
dt_produits.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Nom                   5 non-null      object 
 1   ID Référence produit  5 non-null      object 
 2   Prix                  5 non-null      float64
 3   Stock                 5 non-null      int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 288.0+ bytes


,Prix,Stock
count,5.000000,5.000000
mean,43.990000,85.000000
std,23.021729,26.457513
min,19.990000,50.000000
25%,29.990000,75.000000
50%,39.990000,80.000000
75%,49.990000,100.000000
max,79.990000,120.000000


- Chaque produit est identifié par un identifiant unique (`product_id`)
- Les prix sont numériques et représentent le prix unitaire
- Le stock indique la quantité disponible par produit

In [5]:
# Analyse du dataset ventes
# Afficher les informations générales du dataset ventes
dt_ventes.info()

# Afficher des statistiques descriptives pour les colonnes numériques
dt_ventes.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date                  30 non-null     object
 1   ID Référence produit  30 non-null     object
 2   Quantité              30 non-null     int64 
 3   ID Magasin            30 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 1.1+ KB


,Quantité,ID Magasin
count,30.000000,30.000000
mean,4.066667,3.433333
std,1.799106,1.994533
min,1.000000,1.000000
25%,3.000000,2.000000
50%,4.000000,3.000000
75%,5.000000,5.000000
max,8.000000,7.000000


- Chaque ligne correspond à une vente
- Il n’existe pas d’identifiant unique de vente
- Une vente est définie par une date, un produit et un magasin
- Les quantités vendues sont exprimées en nombre d’unités

#### *Renommer les colonnes pour + de cohérences*

In [6]:
# Renommage des colonnes produits
dt_produits = dt_produits.rename(columns={
    "Nom": "product_name",
    "ID Référence produit": "product_id",
    "Prix": "price",
    "Stock": "stock"
})

# Renommage des colonnes ventes
dt_ventes = dt_ventes.rename(columns={
    "Date": "sale_date",
    "ID Référence produit": "product_id",
    "Quantité": "quantity",
    "ID Magasin": "store_id"
})

# Renommage des colonnes magasins
dt_magasins = dt_magasins.rename(columns={
    "ID Magasin": "store_id",
    "Ville": "city",
    "Nombre de salariés": "employees"
})

Nous avons renommé les colonnes pour ajouter + de clareté aux noms, éviter les caractères spéciaux et faciliter les jointures SQL entre les ventes, produits et les magasins.

#### *Nettoyage & conversion des types*

Dans cette étape, nous allons :
- convertir les colonnes numériques au bon format
- convertir les dates de vente en format date
- s’assurer que les données sont prêtes pour les calculs et le stockage en base

In [ ]:
# Conversion des types produits
dt_produits["price"] = dt_produits["price"].astype(float)
dt_produits["stock"] = dt_produits["stock"].astype(int)

# Conversion des types ventes
dt_ventes["quantity"] = dt_ventes["quantity"].astype(int)
dt_ventes["sale_date"] = pd.to_datetime(dt_ventes["sale_date"])

On a efféctué une conversion des types afin d'éviter les erreurs de calcul et d'avoir des analyses fiables sur le chiffre d'affaires et l'évolution des ventes au cours du temps.

#### *Fusion des datasets*

Dans cette étape, nous allons :
- fusionner les datasets ventes, produits et magasins
- préparer les données pour les analyses SQL et statistiques


In [8]:
# Fusion des ventes avec les produits (via product_id)
df_sales = dt_ventes.merge(
    dt_produits,
    on="product_id",
    how="left"
)

In [9]:
# Fusion avec les magasins (via store_id)
df_sales = df_sales.merge(
    dt_magasins,
    on="store_id",
    how="left"
)

In [10]:
# Aperçu du dataset final
df_sales.head()

,sale_date,product_id,quantity,store_id,product_name,price,stock,city,employees
0,2023-05-27,REF001,5,1,Produit A,49.99,100,Paris,10
1,2023-05-28,REF002,3,2,Produit B,19.99,50,Marseille,5
2,2023-05-29,REF003,2,1,Produit C,29.99,75,Paris,10
3,2023-05-30,REF004,4,3,Produit D,79.99,120,Lyon,8
4,2023-05-31,REF005,7,2,Produit E,39.99,80,Marseille,5


Nous avions effectué une fusion des 3 datasets ventes, produits et magasins afin d'obtenir une vue générale de chaque transaction. Une étape cruciale pour nos futures analyses financières et géographiques.

#### *Calcul du chiffre d'affaire*

Dans cette étape, nous allons calculer le chiffre d'affaire. 
Le chiffre d'affaire est un indicateur clé pour mesurer la performance commerciale : 
- permet d'évaluer les ventes
- il sert de base aux analyses par produit, par magasin et par région

In [ ]:
# Calcul du chiffre d'affaires par ligne de vente
df_sales["revenue"] = df_sales["quantity"] * df_sales["price"]

In [12]:
df_sales[["sale_date", "product_name", "quantity", "price", "revenue"]].head()

,sale_date,product_name,quantity,price,revenue
0,2023-05-27,Produit A,5,49.99,249.95
1,2023-05-28,Produit B,3,19.99,59.97
2,2023-05-29,Produit C,2,29.99,59.98
3,2023-05-30,Produit D,4,79.99,319.96
4,2023-05-31,Produit E,7,39.99,279.93


Le chiffre d'affaires est calculé pour chaque vente en multipliant : la quantité vendue * le prix unitaire du produit

### *Part 2 : Création d'une base SQLite*

Dans cette étapes nous allons procéder à :
- la creéation de la base de données SQLite
- la création des tables nécessaires pour notre analyses financières
- stocker les données nettoyées dans la base
- la préparation des analyses SQL

In [13]:
# Import de la librairie Sqlite3
import sqlite3

#### *Création de la base de données*

In [14]:
# Connexion à la base SQLite
conn = sqlite3.connect("sales_analysis.db")

# Création d'un curseur pour exécuter les requêtes SQL
cursor = conn.cursor()

Une base de données SQLite nommée `sales_analysis.db` est créée localement. Elle sera utilisée pour stocker les données et exécuter les requêtes SQL.

#### *Création des Tables*

In [15]:
# Création de la table produits
cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    price REAL,
    stock INTEGER
)
""")

# Création de la table magasins
cursor.execute("""
CREATE TABLE IF NOT EXISTS stores (
    store_id INTEGER PRIMARY KEY,
    city TEXT,
    employees INTEGER
)
""")

# Création de la table ventes
cursor.execute("""
CREATE TABLE IF NOT EXISTS sales (
    sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
    sale_date TEXT,
    product_id INTEGER,
    store_id INTEGER,
    quantity INTEGER,
    revenue REAL,
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (store_id) REFERENCES stores(store_id)
)
""")

# Sauvegarde des changements
conn.commit()

Les tables produits, magasins et ventes ont été créées. Les relations entre les tables sont définies via des clés étrangères.


#### *Insertion des données dans SQLite*

In [16]:
# Insertion des produits dans la table products
dt_produits.to_sql(
    "products",
    conn,
    if_exists="replace",
    index=False
)

5

In [17]:
# Insertion des magasins dans la table stores
dt_magasins.to_sql(
    "stores",
    conn,
    if_exists="replace",
    index=False
)

7

In [18]:
# Sélection des colonnes nécessaires pour la table sales
df_sales_sql = df_sales[[
    "sale_date",
    "product_id",
    "store_id",
    "quantity",
    "revenue"
]]

# Insertion des ventes
df_sales_sql.to_sql(
    "sales",
    conn,
    if_exists="replace",
    index=False
)

30

Les données ont été insérées dans la base SQLite.
La base est maintenant prête pour l’exécution de requêtes SQL d’analyse.


### *Part 3 : Analyses SQL des Ventes*

Dans cette étape, nous allons éxecuter des requêtes SQL afin de :
- calculer le chiffre d'affaire total
- analyser les ventes par produit
- analyser les ventes par ville

#### *Chiffre d'affaires total*

In [19]:
# Requête SQL pour calculer le chiffre d'affaires total

query_total_revenue = """
SELECT SUM(revenue) AS total_revenue
FROM sales
"""

In [20]:
# Exécution de la requête et affichage du résultat

total_revenue = pd.read_sql_query(query_total_revenue, conn)
total_revenue

,total_revenue
0,5268.78


Le chiffre d'affaire total correspond à la somme des revenus générés par l'ensemble des ventes sur la période étudiée.

#### *Ventes par produit*

In [21]:
# Requête SQL pour obtenir les ventes par produit

query_sales_by_product = """
SELECT 
    p.product_name,
    SUM(s.quantity) AS total_quantity,
    SUM(s.revenue) AS total_revenue
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name
ORDER BY total_revenue DESC
"""

In [22]:
# Exécution de la requête

sales_by_product = pd.read_sql_query(query_sales_by_product, conn)
sales_by_product

,product_name,total_quantity,total_revenue
0,Produit D,21,1679.79
1,Produit E,35,1399.65
2,Produit A,24,1199.76
3,Produit B,27,539.73
4,Produit C,15,449.85


Cette requête nous a permis d'identifier : 
 - les produits les plus vendus
 - les produits générant le plus de chiffre d'affaires

#### *Ventes par ville*

In [23]:
# Requête SQL pour analyser les ventes par ville

query_sales_by_city = """
SELECT 
    st.city,
    SUM(s.quantity) AS total_quantity,
    SUM(s.revenue) AS total_revenue
FROM sales s
JOIN stores st ON s.store_id = st.store_id
GROUP BY st.city
ORDER BY total_revenue DESC
"""

In [24]:
# Exécution de la requête

sales_by_city = pd.read_sql_query(query_sales_by_city, conn)
sales_by_city

,city,total_quantity,total_revenue
0,Lyon,21,1059.79
1,Marseille,27,1009.73
2,Bordeaux,19,829.81
3,Paris,20,799.80
4,Nantes,17,739.83
5,Strasbourg,11,579.89
6,Lille,7,249.93


Cette analyse permet de comparer les performances commerciales
des différentes villes dans lesquelles l’entreprise est implantée.

En résumé, les analyses réalisées ont permis de :
- calculer le chiffre d’affaires global
- identifier les produits les plus performants
- analyser la répartition géographique des ventes

Ces résultats sont déterminantes pour la prise de décision commerciale.


### *Part 4 : Stockage des résultats d'analyse dans SQLite*

Dans cette étape, nous allons : 
 - créer des tables dédiées aux résultats d'analyse
 - stocker les résultats des requêtes SQL
 - conserver l'historique des analyses

#### *Création des tables de résultats*

In [25]:
# Table pour le chiffre d'affaires total

cursor.execute("""
CREATE TABLE IF NOT EXISTS analysis_total_revenue (
    analysis_date TEXT,
    total_revenue REAL
)
""")

# Table pour les ventes par produit
cursor.execute("""
CREATE TABLE IF NOT EXISTS analysis_sales_by_product (
    product_name TEXT,
    total_quantity INTEGER,
    total_revenue REAL
)
""")

# Table pour les ventes par ville
cursor.execute("""
CREATE TABLE IF NOT EXISTS analysis_sales_by_city (
    city TEXT,
    total_quantity INTEGER,
    total_revenue REAL
)
""")

# Sauvegarde des changements
conn.commit()

Des tables spécifiques ont été créées afin de stocker les résultats
des analyses SQL.

#### *Insertion des résultats dans SQLite*

In [26]:
# Ajout de la date d'analyse

total_revenue["analysis_date"] = pd.Timestamp.now().strftime("%Y-%m-%d")

# Insertion dans la table SQLite
total_revenue.to_sql(
    "analysis_total_revenue",
    conn,
    if_exists="append",
    index=False
)

1

Le chiffre d’affaires total est stocké avec la date d’analyse
afin de conserver un historique des résultats.


#### *Stockage des ventes par produit*

In [27]:
# Insertion des ventes par produit

sales_by_product.to_sql(
    "analysis_sales_by_product",
    conn,
    if_exists="replace",
    index=False
)

5

#### *Stockage des ventes par ville*

In [28]:
# Insertion des ventes par ville

sales_by_city.to_sql(
    "analysis_sales_by_city",
    conn,
    if_exists="replace",
    index=False
)

7

#### *Vérification du fichier sales_analysis.db*

In [ ]:
# connexion à la base de données
# on ouvre le fichier sales_analysis.db
# conn représente la connexion active à la base
conn = sqlite3.connect("sales_analysis.db")

# on execute une requête SQL sur la base SQLite afin d'afficher la liste de toutes les tables existantes
pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)

,name
0,sqlite_sequence
1,analysis_total_revenue
2,products
3,stores
4,sales
5,analysis_sales_by_product
6,analysis_sales_by_city


In [ ]:
# on exécute une requête SQL pour afficher le contenu complet de la table analysis_sales_by_city sous forme de df pandas
pd.read_sql_query(
    "SELECT * FROM analysis_sales_by_city;",
    conn
)

,city,total_quantity,total_revenue
0,Lyon,21,1059.79
1,Marseille,27,1009.73
2,Bordeaux,19,829.81
3,Paris,20,799.80
4,Nantes,17,739.83
5,Strasbourg,11,579.89
6,Lille,7,249.93


En conclusion, le fichier sales_analysis.db a bien été crée.

On peut ouvrir la base SQLite directement depuis python pour vérifier les tables et les résultats d'analyse obtenus.